In [2]:
Id=[]
import numpy as np 
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/kaggle/input/vegetation-diseases-detection/train'):
    for filename in filenames:
        Id.append(os.path.join(dirname, filename))

In [3]:
train=pd.DataFrame()
train=train.assign(filename=Id)
train.head()

,filename


In [4]:
train['label']=train['filename']
train['label']=train['label'].str.replace('/kaggle/input/vegetation-diseases-detection/train/','')
train.head()

AttributeError: Can only use .str accessor with string values!

In [5]:
train=train[train['label'].str.contains("Tomato")]
train.reset_index(inplace=True,drop=True)
train.head()

AttributeError: Can only use .str accessor with string values!

In [ ]:
count=(train.label.value_counts()).to_frame()
count.reset_index(inplace=True)
count.columns=['label','count']
count

In [ ]:
from sklearn.model_selection import train_test_split
train,test= train_test_split( train, test_size=0.2, random_state=42,shuffle=True, stratify=train['label'])

In [ ]:
count=(train.label.value_counts()).to_frame()
count.reset_index(inplace=True)
count.columns=['label','count']
count